# Ver 1 using char-cnn model

In [1]:
from theano.sandbox import cuda
%matplotlib inline
from __future__ import division, print_function
import operator

import numpy as np
from keras.layers import TimeDistributed, Activation
from numpy.random import choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from numpy.random import choice
import time

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
text = open('./data/lgqm-sample.txt').read()
text = unicode(text, "utf-8")

In [3]:
print('length of text: ', len(text), text[:100])

length of text:  35397 《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何


In [4]:
charset = set(text)
char_freq_map = {c: 0 for c in charset}

In [5]:
for c in text:
    char_freq_map[c] += 1

In [6]:
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]
print('size of charset', len(charlist), charlist[:40])

size of charset 2064 [u' ', u'\uff0c', u'\u7684', u'\r', u'\n', u'\u3002', u'\u662f', u'\u4e00', u'\u4e0d', u'\u4e86', u'\u8fd9', u'\u4eba', u'\u4e2a', u'\u201c', u'\u201d', u'\u5b50', u'\u6709', u'\u6765', u'\u4ed6', u'\u5728', u'\u9ad8', u'\u5927', u'\u4e5f', u'\u4eec', u'\u91cc', u'\u4e0a', u'\u90fd', u'\u8001', u'\u8fc7', u'\u5230', u'\u5f97', u'\u5c31', u'\u8bf4', u'\u591a', u'\u7237', u'\u4e0b', u'\u4f1a', u'\u4e4b', u'\u6d77', u'\u6587']


In [7]:
char2idx = dict((c, i) for i, c in enumerate(charlist))

In [8]:
text_in_idx = [char2idx[c] for c in text]

In [9]:
#text_in_idx[:6]
print(''.join(charlist[i] for i in text_in_idx[:24]))

《临高启明》吹牛者

严正声明：本书为丫丫小


In [10]:
maxlen = 48
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 35349


In [11]:
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape

((35349, 48), (35349, 48))

In [12]:
n_fac = 200
batch_size = 64
n_hidden = 512
vocab_size = len(charlist)

In [13]:
# stateless with batch
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen, batch_input_shape=(batch_size, maxlen)),
        BatchNormalization(),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
# non-stateful?
# model=Sequential([
#         Embedding(vocab_size, n_fac, input_length=maxlen),
#         BatchNormalization(),
#         LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size)),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size, activation='softmax')),
#     ])    

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (64, 48, 200)         412800      embedding_input_1[0][0]          
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(64, 48, 200)         400         embedding_1[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (64, 48, 512)         1460224     batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dropout_1 (Dropout)              (64, 48, 512)         0           lstm_1[0][0]                     
___________________________________________________________________________________________

In [29]:
def run_epochs(n):
    # input must be exact multiples of batch_size, for stateful
    multiples = len(sentences)//batch_size*batch_size
    sentences_truncated = sentences[:multiples]
    next_chars_truncated = next_chars[:multiples]
    next_chars_adjusted = np.expand_dims(next_chars_truncated,-1)
    for i in range(n):
        model.reset_states()
        h = model.fit(sentences_truncated, next_chars_adjusted, batch_size=batch_size, nb_epoch=1, shuffle=True)
        print(h.history, h.history['loss'])
        weight_file_name = './data/text-generation-lgqm-ver-1-char-cnn-'+str(time.time())+'.h5'
        model.save_weights(weight_file_name)
        print('saved weights: ', weight_file_name)

In [30]:
# NOTE: when cr is not together with lf, a line is biten! But, I doubt it ever happens in original text
# Better learn more to let it avoid naturally

def print_example_inline_seed_non_stateful():
    pre_string = u"第65535节 "
    
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen]
    
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    print(seed_string)
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print('final: ', seed_string)

In [31]:
model.optimizer.lr=0.01
# model.load_weights('./data/text-generation-lgqm-ver-1-char-cnn-1497597351.55.h5')

In [33]:
run_epochs(1)

Epoch 1/1
35328/35328 [==============================] - 161s - loss: 1.2177   
{'loss': [1.2177366734198902]} [1.2177366734198902]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497598592.5.h5


In [34]:
print_example_inline_seed_non_stateful()

第65535节 记不清了――他照旧属于要庆祝11.11的那伙人。[搜索最新更新尽在]

    一伙古装的士兵
final:  第65535节 记不清了――他照旧属于要庆祝11.11的那伙人。[搜索最新更新尽在]

    一伙古装的士兵，在骑马的军官带领下，把十多个乡民模样的衣服。文总也算是经过些风月场面的人物。

    管事领着家丁们从栈房院子里搬来搬。萧子山不由得哑然一笑，这年头的看着他们。

    “子山。一会我发这几个倭人把你吓得心里做？”萧子山严重的表示置疑。

    至于钱，虽然他们心中揣度。莫非是日本国内哪位名主地使者？转念一想。倭人自己日用尚且不足。百货十多少白银的收购了许多大小款式不一的镜子、一次性打火机、各种磨砂、喷花的玻璃杯，居然惨败！文不及这几个孩子又是能吃的时候，一家人靠七钱银子开销，着实有点吃力。

    临走之前，文德嗣留下二百两银子，要高老爷在临近购置疑。[搜索最新更新尽在]

    临高就成了最好的选择，他们拒绝了更省油的电喷引擎的新款，买了一辆老款的切诺基吉普：四驱的手推车。

    议会民主？三权分立？君主立宪？集权主义？民族主义？社会主义？社会主义？社会主义？[搜索最新更新尽在]

    虽然文德嗣也在网络上看过一些相当血淋淋的照片。血肉模糊的无头尸体在900万像素，单反镜头的演绎下逼真得让人想吐。

    萧子山毕业以后，在珠三角的几家企业都呆过，上过黑心老板的当，做过不切实际的梦，最后好歹在一家外企的找了份待遇还不错的工作，勤勤恳恳的干活拿工资――也许多所老爷的脸色，连平日里最得宠的小妾，都懒得临幸了。

    传统穿越暴利神器玻璃杯，居然惨败！一千两成了一两，这个落差也是在太大了，钻石变成了玻璃。

    玻璃杯，每个一两。

第八点m/
    “对，很有可能。”

    “是啊，这才是男人的天堂啊！”

    货款按他们谈好的价格，结算下来，共得白银三千多两白银，其中几面全身穿衣镜，看得高老爷眼睛都快掉下来了。

    “不过，敝人倒是另有一个主意……”三个人吃了一惊，欠起


In [20]:
# for i in range(6):
#     run_epochs(20)
#     print_example_inline_seed_non_stateful()

In [37]:
# use entire text
text = open('./data/lgqm.txt').read()
text = unicode(text, "utf-8")
print('length of text: ', len(text), text[:100])
charset = set(text)
char_freq_map = {c: 0 for c in charset}
for c in text:
    char_freq_map[c] += 1
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]
print('size of charset', len(charlist))
for i in range(40):
    print(charlist[i])
char2idx = dict((c, i) for i, c in enumerate(charlist))
text_in_idx = [char2idx[c] for c in text]
#text_in_idx[:24]
print(''.join(charlist[i] for i in text_in_idx[:24]))
maxlen = 48
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape
print('xxx done')

length of text:  9215340 《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何
size of charset 5688
 
，
的



。
是
了
一
不
“
”
这
人
有
在
他
来
个
大
上
就
得
到
要
们
和
说
里
―
也
着
子
下
还
出
过
地
会
道
《临高启明》吹牛者

严正声明：本书为丫丫小
nb sequences: 9215292


((9215292, 48), (9215292, 48))

In [39]:
n_fac = 200
batch_size = 64
n_hidden = 512
vocab_size = len(charlist)
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen, batch_input_shape=(batch_size, maxlen)),
        BatchNormalization(),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (64, 48, 200)         1137600     embedding_input_2[0][0]          
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(64, 48, 200)         400         embedding_2[0][0]                
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (64, 48, 512)         1460224     batchnormalization_2[0][0]       
____________________________________________________________________________________________________
dropout_4 (Dropout)              (64, 48, 512)         0           lstm_3[0][0]                     
___________________________________________________________________________________________

In [40]:
run_epochs(1)

Epoch 1/1
7154944/9215232 [======================>.......] - ETA: 13255s - loss: 3.5622

KeyboardInterrupt: 